## TumbuhKuy Condition Classification

### 0. Importiing dependencies and dataset

In [3]:
# data handling
import pandas as pd

# read dataset
dataset_gizi = pd.read_csv("data_klasifikasi_gizi.csv")
dataset_stunting = pd.read_csv("data_klasifikasi_stunting.csv")

In [21]:
dataset_gizi.tail()

,Gender,Tahun,Bulan,-3 SD,-2 SD,-1 SD,Median,+1 SD,+2 SD,+3 SD
453,Laki-laki,18,8,15.8,17.5,19.5,22.0,25.3,29.6,35.5
454,Laki-laki,18,9,15.8,17.5,19.5,22.1,25.3,29.6,35.5
455,Laki-laki,18,10,15.8,17.5,19.6,22.1,25.4,29.6,35.5
456,Laki-laki,18,11,15.8,17.5,19.6,22.2,25.4,29.7,35.5
457,Laki-laki,19,0,15.9,17.6,19.6,22.2,25.4,29.7,35.5


In [22]:
dataset_stunting.tail()

,Gender,Tahun,Bulan,-3 SD,-2 SD,-1 SD,Median,+1 SD,+2 SD,+3 SD
117,Laki-laki,4,8,94.3,98.8,103.3,107.8,112.3,116.7,121.2
118,Laki-laki,4,9,94.7,99.3,103.8,108.3,112.8,117.4,121.9
119,Laki-laki,4,10,95.2,99.7,104.3,108.9,113.4,118.0,122.6
120,Laki-laki,4,11,95.6,100.2,104.8,109.4,114.0,118.6,123.2
121,Laki-laki,5,0,96.1,100.7,105.3,110.0,114.6,119.2,123.9


### 1. Make the categorization code

In [ ]:
# code to get the clasification
year_age: int = 16
month_age: int = 0

is_female: bool = True

if is_female: gender = "Perempuan"
else: gender = "Laki-laki"

massa_tubuh: float = 14.6

selected_category = dataset_gizi.loc[(dataset_gizi["Gender"] == gender) & 
                                (dataset_gizi["Tahun"] == year_age) &
                                (dataset_gizi["Bulan"] == month_age)]

top = massa_tubuh - selected_category["Median"].item()
if top >= 0:
    z_score = top / (selected_category["+1 SD"].item() - selected_category["Median"].item())
else:
    z_score = top / (selected_category["Median"].item() - selected_category["-1 SD"].item())

print("Median:", selected_category["Median"].item())
print("Z score", z_score)

# z score classification
if z_score < -3: print("severly thiness")
elif z_score < -2: print("thiness")
elif z_score < 1: print("normal")
elif z_score < 2: print("overweight")
else: print("obese")

Median: 20.7
Z score -2.44
thiness


### Going modular

In [ ]:
class StatusClassifier:
    def __init__(self, data_gizi: str = "data_klasifikasi_gizi.csv", data_stunting: str = "data_klasifikasi_stunting.csv") -> None:
        self.dataset_gizi: pd.DataFrame = pd.read_csv(data_gizi)
        self.dataset_stunting: pd.DataFrame = pd.read_csv(data_stunting)
        self.year_age: int = 0
        self.month_age: int = 0
        self.gender: str = ""
        self.status: str = ""

    def set_category(self, gender: str, month_age: int = 0, year_age: int = 0):
        self.gender = gender
        self.month_age = month_age
        self.year_age = year_age
        
    def get_classification(self, mode: str = "gizi", massa_tubuh: float = None, tinggi_tubuh: float = None):
        if mode == "gizi": # mode gizi
            if self.year_age >= 0 and self.year_age <= 19:
                self.selected_classifier = self.dataset_gizi.loc[(self.dataset_gizi["Gender"] == self.gender) & 
                                                                (self.dataset_gizi["Tahun"] == self.year_age) &
                                                                (self.dataset_gizi["Bulan"] == self.month_age)]
                
                top_equation = massa_tubuh - self.selected_classifier["Median"].item()

                if top_equation >= 0:
                    z_score = top_equation / (self.selected_classifier["+1 SD"].item() - self.selected_classifier["Median"].item())
                else:
                    z_score = top_equation / (self.selected_classifier["Median"].item() - self.selected_classifier["-1 SD"].item())

                if z_score < -3: self.status = "Severly Thiness"
                elif z_score < -2: self.status = "Thiness"
                elif z_score < 1: self.status = "Normal"
                elif z_score < 2: self.status = "Overweight"
                else: self.status = "Obese"
            else: return "Wrong input values"
        
        else: # mode stunting  
            if self.year_age >= 0 and self.year_age <= 5:       
                self.selected_classifier = self.dataset_stunting.loc[(self.dataset_stunting["Gender"] == self.gender) & 
                                                                    (self.dataset_stunting["Tahun"] == self.year_age) &
                                                                    (self.dataset_stunting["Bulan"] == self.month_age)]
                
                top_equation = tinggi_tubuh - self.selected_classifier["Median"].item()

                if top_equation >= 0:
                    z_score = top_equation / (self.selected_classifier["+1 SD"].item() - self.selected_classifier["Median"].item())
                else:
                    z_score = top_equation / (self.selected_classifier["Median"].item() - self.selected_classifier["-1 SD"].item())

                if z_score < -3: self.status = "Severly Stunted"
                elif z_score < -2: self.status = "Stunted"
                elif z_score < 3: self.status = "Normal"
                else: self.status = "Tinggi"
            else: return "Wrong input values"

        return self.status

In [26]:
classifier = StatusClassifier()
classifier.set_category("Laki-laki", 0, 14)

print(classifier.get_classification(mode="gizi", massa_tubuh=massa_tubuh))
print(classifier.get_classification(mode="stunting", tinggi_tubuh=150))

Obese
Wrong input values
